# ETL - Base de Datos Cámara de Comercio de Ibagué

## Proceso de Extracción, Transformación y Carga (ETL)

**Proyecto**: Sistema Web de Análisis Financiero con Machine Learning para PYMES de Ibagué, Tolima

**Objetivo de este notebook**: Explorar, limpiar y validar la base de datos de empresas activas de la Cámara de Comercio de Ibagué, preparar los datasets SIREM de la Superintendencia de Sociedades, y realizar el cruce (JOIN) por NIT entre ambas fuentes para identificar las PYMES de Ibagué con datos financieros disponibles.

**Fuentes de datos**:
- [SIREM - Superintendencia de Sociedades](https://www.datos.gov.co/) (~9.17 GB en 4 archivos CSV)
- [Cámara de Comercio de Ibagué](https://www.datos.gov.co/) (23.58 MB, 1 archivo CSV)

**Estrategia de integración**: Cruce mediante el campo NIT (Número de Identificación Tributaria) para filtrar únicamente las empresas registradas en Ibagué que además reportan estados financieros al SIREM.

In [29]:
# Importacion de librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import re

# Configuracion general
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 60)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 100

print('Librerias cargadas correctamente')

Librerias cargadas correctamente


---
## 1. Carga del Dataset - Cámara de Comercio de Ibagué

La base de datos de la Cámara de Comercio contiene el registro de todas las empresas y entidades activas en la jurisdicción de Ibagué, con corte a 31 de diciembre de 2025. Este dataset es relativamente pequeño (23.58 MB) y se carga completamente en memoria.

In [30]:
BASE_PATH = Path(r'C:\Users\USUARIO1\Documents\Tesis\dataset')
ARCHIVO_CAMARA = 'BASE_DE_DATOS_DE_EMPRESAS_Y_O_ENTIDADES_ACTIVAS_-_JURISDICCIÓN_CÁMARA_DE_COMERCIO_DE_IBAGUÉ_-_CORTE_A_31_DE_DICIEMBRE_DE_2025_20260207.csv'

ruta = BASE_PATH / ARCHIVO_CAMARA
print(f'Archivo: {ARCHIVO_CAMARA}')
print(f'Tamaño: {ruta.stat().st_size / (1024**2):.2f} MB')

df_camara = pd.read_csv(ruta, encoding='utf-8', low_memory=False)

print(f'\nRegistros: {len(df_camara):,}')
print(f'Columnas: {len(df_camara.columns)}')

Archivo: BASE_DE_DATOS_DE_EMPRESAS_Y_O_ENTIDADES_ACTIVAS_-_JURISDICCIÓN_CÁMARA_DE_COMERCIO_DE_IBAGUÉ_-_CORTE_A_31_DE_DICIEMBRE_DE_2025_20260207.csv
Tamaño: 23.58 MB

Registros: 20,280
Columnas: 85


In [31]:
print('ESTRUCTURA DEL DATASET')
print('=' * 60)
print(f'Dimensiones: {df_camara.shape[0]:,} filas x {df_camara.shape[1]} columnas')
print(f'\nTipos de datos:')
print(df_camara.dtypes.to_string())

ESTRUCTURA DEL DATASET
Dimensiones: 20,280 filas x 85 columnas

Tipos de datos:
MATRICULA                                                   object
PROPONENTE                                                  object
ORGANIZACION                                                object
CATEGORIA                                                   object
ESTADO DE LA MATRICULA                                      object
RAZON SOCIAL                                                object
EMPRENDIMIENTO SOCIAL                                       object
NIT                                                         object
NACIONALIDAD                                                object
FECHA DE MATRICULA                                         float64
FECHA RENOVACION                                           float64
ULTIMO AÑO RENOVADO                                        float64
FECHA CONSTITUCION                                          object
FECHA DE VIGENCIA                                

In [32]:
display(df_camara.head(5))

,MATRICULA,PROPONENTE,ORGANIZACION,CATEGORIA,ESTADO DE LA MATRICULA,RAZON SOCIAL,EMPRENDIMIENTO SOCIAL,NIT,NACIONALIDAD,FECHA DE MATRICULA,FECHA RENOVACION,ULTIMO AÑO RENOVADO,FECHA CONSTITUCION,FECHA DE VIGENCIA,DIRECCION COMERCIAL,BARRIO COMERCIAL,MUNICIPIO COMERCIAL,VIGILANCIA,EMAIL COMERCIAL 1,EMAIL COMERCIAL 2,EMAIL COMERCIAL 3,DIRECCION DE NOTIFICACION,MUNICIPIO DE NOTIFICACION,EMAIL DE NOTIFICACION,CIIU-1,CIIU-2,CIIU-3,CIIU-4,ACTIVIDAD,CANTIDAD DE MUJERES,CANTIDAD DE MUJERES EN CARGOS DIRECTIVOS,% DE PARTICIPACIÓN DE MUJERES,CIIU TAMAÑO EMPRESARIAL,AÑO DATOS TAMAÑO EMPRESARIAL,FECHA DATOS TAMAÑO EMPRESARIAL,TAMAÑO EMPRESA,LIBROS DE COMERCIO,CTR. EMBARGO,IMPORTA / EXPORTA,TIPO LOCAL SEDE ADMINISTRATIVA,TIEMPO FUNCIONAMIENTO,UBICACION,CLASE GENERAL DE ESADL,CLASE ESPECIAL ESADL,CLASE DE ECONOMIA SOLIDARIA,BENEFICIARIO ART. 7 LEY 1429,BENEFICIARIO LEY 1780 - MATRICULA,CUMPLE LEY 1780 RENOVACION,MANTIENE BENEFICIOS LEY 1780,RENUNCIA BENEFICIOS LEY 1780,TIPO DE PROPIETARIO,BIC - LEY 1901 DE 2018,PERSONAL,GRUPO NIIF,AÑO DE DATOS,FECHA DE DATOS NIIF,CANTIDAD DE ESTABLECIMIENTOS,CANTIDAD DE AGENCIAS /SUCURSALES,PORCENTAJE COMPOSICION DEL CAPITAL NACIONAL PRIVADO,PORCENTAJE COMPOSICION DEL CAPITAL NACIONAL PUBLICO,PORCENTAJE COMPOSICION DEL CAPITAL NACIONAL TOTAL,PORCENTAJE COMPOSICION DEL CAPITAL EXTRANJERO PRIVADO,PORCENTAJE COMPOSICION DEL CAPITAL EXTRANJERO PUBLICO\t,PORCENTAJE COMPOSICION DEL CAPITAL EXTRANJERO TOTAL,MATRICULA PROPIETARIO,NIT PROPIETARIO,CAMARA DE PROPIETARIO,NOMBRE DE PROPIETARIO,DIRECCION PROPIETARIO,MUNICIPIO PROPIETARIO,EMAIL PROPIETARIO,FECHA DE PAGO DE RENOVACION 2016,FECHA DE PAGO DE RENOVACION 2017,FECHA DE PAGO DE RENOVACION 2018,FECHA DE PAGO DE RENOVACION 2019,FECHA DE PAGO DE RENOVACION 2020,FECHA DE PAGO DE RENOVACION 2021,FECHA DE PAGO DE RENOVACION 2022,FECHA DE PAGO DE RENOVACION 2023,FECHA DE PAGO DE RENOVACION 2024,FECHA DE PAGO DE RENOVACION 2025,NUMERO DE SOCIOS,REPRESENTANTE LEGAL SUPLENTE,REPRESENTANTE LEGAL,FECHA DE LA INFORMACION
0,S0507787,No aplica,ESAL,PRINCIPAL,IA,FUNDACION MISIONERA VOLVIENDO A CASA,N,No aplica,No aplica,"20,251,230.00","20,251,230.00","2,025.00",No aplica,No aplica,CLL 15 No. 3A-28,01008 - BRR CENTRO,73001 - IBAGUE,ASUNTO DELEGADOS DE LA NACIÓN,funcasa.org@gmail.com,No reporta,No reporta,CLL 15 No. 3A-28,73001 - IBAGUE,funcasa.org@gmail.com,S9499 ** Actividades de otras asociaciones n.c.p.,No reporta,No reporta,No reporta,No reporta,0,0,0,S9499,2025,20251230,MICRO EMPRESA,No reporta,N,No reporta,No reporta,No reporta,2,No aplica,63 - FUNDACIONES,No aplica,N,N,N,No aplica,No aplica,No aplica,No reporta,0.00,GRUPO III - MICROEMPRESAS,2025,20251229,No reporta,No reporta,0.00,0.00,0.00,0.00,0.00,0.00,No aplica,No aplica,No aplica,No aplica,No reporta,No reporta,No reporta,No reporta,No reporta,No reporta,No reporta,No reporta,No reporta,No reporta,No reporta,No reporta,No reporta,0.00,No reporta,No reporta,2025 Dec 31 12:00:00 AM
1,S0507786,No aplica,ESAL,PRINCIPAL,IA,ASOCIACION DE EXTENSIONISTAS DEL TOLIMA,N,No aplica,No aplica,"20,251,230.00","20,251,230.00","2,025.00",No aplica,No aplica,CR 13 No. 29A-11,03016 - BRR ANTONIO NARINO,73001 - IBAGUE,ASUNTOS DELEGADOS DE LA NACION,ednaruthnavarro@hotmail.com,No reporta,No reporta,CR 13 No. 29A-11,73001 - IBAGUE,ednaruthnavarro@hotmail.com,S9499 ** Actividades de otras asociaciones n.c.p.,No reporta,No reporta,No reporta,No reporta,1,1,0,S9499,2025,20251230,MICRO EMPRESA,No reporta,N,No reporta,No reporta,No reporta,5,No aplica,23 - ASOCIACIONES MUTUALES,No aplica,N,N,N,No aplica,No aplica,No aplica,No reporta,8.00,GRUPO III - MICROEMPRESAS,2025,20251212,No reporta,No reporta,0.00,0.00,0.00,0.00,0.00,0.00,No aplica,No aplica,No aplica,No aplica,No reporta,No reporta,No reporta,No reporta,No reporta,No reporta,No reporta,No reporta,No reporta,No reporta,No reporta,No reporta,No reporta,0.00,No reporta,No reporta,2025 Dec 31 12:00:00 AM
2,S0507785,No aplica,ESAL,PRINCIPAL,IA,FUNDACION ANGOVA,S,No aplica,No aplica,"20,25

---
## 2. Análisis de Calidad de Datos

### 2.1 Valores Nulos (NaN)

Verificamos la presencia de valores nulos reales (`NaN`) en cada columna del dataset.

In [33]:
nulos = df_camara.isnull().sum()
pct_nulos = (nulos / len(df_camara) * 100).round(2)

df_nulos = pd.DataFrame({
    'Nulos': nulos,
    'Porcentaje (%)': pct_nulos
}).sort_values('Nulos', ascending=False)

df_nulos_con_datos = df_nulos[df_nulos['Nulos'] > 0]

print(f'Columnas con valores nulos: {len(df_nulos_con_datos)} de {len(df_camara.columns)}')
print('=' * 60)
display(df_nulos_con_datos)

Columnas con valores nulos: 84 de 85


,Nulos,Porcentaje (%)
VIGILANCIA,9,0.04
CLASE GENERAL DE ESADL,5,0.02
MATRICULA PROPIETARIO,5,0.02
PORCENTAJE COMPOSICION DEL CAPITAL EXTRANJERO PUBLICO\t,5,0.02
PORCENTAJE COMPOSICION DEL CAPITAL EXTRANJERO PRIVADO,5,0.02
PORCENTAJE COMPOSICION DEL CAPITAL NACIONAL TOTAL,5,0.02
PORCENTAJE COMPOSICION DEL CAPITAL NACIONAL PUBLICO,5,0.02
PORCENTAJE COMPOSICION DEL CAPITAL NACIONAL PRIVADO,5,0.02
CANTIDAD DE AGENCIAS /SUCURSALES,5,0.02
CANTIDAD DE ESTABLECIMIENTOS,5,0.02


### 2.2 Valores "No aplica" y "No reporta" (Nulos Disfrazados)

El dataset utiliza las cadenas **"No aplica"** y **"No reporta"** como sustitutos de valores nulos en las columnas de tipo texto. Estos representan información que no fue proporcionada o que no corresponde al tipo de registro. Es fundamental identificarlos porque afectan el conteo real de datos disponibles.

In [34]:
VALORES_NULOS = ['No aplica', 'No reporta', 'no aplica', 'no reporta', '']

print('VALORES "No aplica" / "No reporta" POR COLUMNA')
print('=' * 60)

for col in df_camara.columns:
    if df_camara[col].dtype == 'object':
        conteo_na = df_camara[col].isin(VALORES_NULOS).sum()
        if conteo_na > 0:
            pct = conteo_na / len(df_camara) * 100
            print(f'{col}: {conteo_na:,} ({pct:.1f}%)')

VALORES "No aplica" / "No reporta" POR COLUMNA
PROPONENTE: 19,887 (98.1%)
EMPRENDIMIENTO SOCIAL: 9,200 (45.4%)
NIT: 9,146 (45.1%)
NACIONALIDAD: 12,839 (63.3%)
FECHA CONSTITUCION: 15,533 (76.6%)
FECHA DE VIGENCIA: 16,610 (81.9%)
BARRIO COMERCIAL: 313 (1.5%)
VIGILANCIA: 18,299 (90.2%)
EMAIL COMERCIAL 1: 6 (0.0%)
EMAIL COMERCIAL 2: 20,272 (100.0%)
EMAIL COMERCIAL 3: 20,275 (100.0%)
DIRECCION DE NOTIFICACION: 7,744 (38.2%)
MUNICIPIO DE NOTIFICACION: 7,744 (38.2%)
EMAIL DE NOTIFICACION: 8,115 (40.0%)
CIIU-2: 12,910 (63.7%)
CIIU-3: 16,958 (83.6%)
CIIU-4: 18,620 (91.8%)
ACTIVIDAD: 3,699 (18.2%)
CANTIDAD DE MUJERES: 11 (0.1%)
CANTIDAD DE MUJERES EN CARGOS DIRECTIVOS: 11 (0.1%)
% DE PARTICIPACIÓN DE MUJERES: 11 (0.1%)
CIIU TAMAÑO EMPRESARIAL: 9,157 (45.2%)
AÑO DATOS TAMAÑO EMPRESARIAL: 9,129 (45.0%)
FECHA DATOS TAMAÑO EMPRESARIAL: 9,145 (45.1%)
TAMAÑO EMPRESA: 9,137 (45.1%)
LIBROS DE COMERCIO: 18,826 (92.8%)
CTR. EMBARGO: 7 (0.0%)
IMPORTA / EXPORTA: 20,217 (99.7%)
TIPO LOCAL SEDE ADMINISTRATIVA

---
## 3. Análisis del Campo NIT

El NIT (Número de Identificación Tributaria) es el campo clave para realizar el cruce con los datasets SIREM. Es fundamental entender su formato, detectar valores inválidos y prepararlo para el JOIN.

In [35]:
print('DIAGNOSTICO DEL CAMPO NIT')
print('=' * 60)

print(f'Tipo de dato: {df_camara["NIT"].dtype}')
print(f'Total registros: {len(df_camara):,}')
print(f'Valores unicos: {df_camara["NIT"].nunique():,}')
print(f'Nulos reales (NaN): {df_camara["NIT"].isnull().sum():,}')

nit_no_aplica = df_camara['NIT'].isin(VALORES_NULOS).sum()
print(f'Valores "No aplica"/"No reporta": {nit_no_aplica:,}')

print(f'\nDistribucion de valores mas frecuentes:')
print(df_camara['NIT'].value_counts().head(15))

DIAGNOSTICO DEL CAMPO NIT
Tipo de dato: object
Total registros: 20,280
Valores unicos: 11,127
Nulos reales (NaN): 5
Valores "No aplica"/"No reporta": 9,146

Distribucion de valores mas frecuentes:
NIT
No aplica      9146
11105765067       2
11056872419       2
8600073354        2
9007305853        1
11104961551       1
657805010         1
516574657         1
11105303055       1
287992506         1
11105299930       1
943922443         1
11105681458       1
9019398148        1
934065977         1
Name: count, dtype: int64


In [36]:
# Excluir registros sin NIT valido
nits_con_valor = df_camara[~df_camara['NIT'].isin(VALORES_NULOS)]['NIT'].astype(str)

# Detectar presencia de caracteres especiales
print('ANALISIS DE FORMATO DEL NIT')
print('=' * 60)
print(f'Registros con NIT informado: {len(nits_con_valor):,}')
print(f'Solo digitos: {nits_con_valor.str.match(r"^\\d+$", na=False).sum():,}')
print(f'Con comas: {nits_con_valor.str.contains(",", na=False).sum():,}')
print(f'Con puntos: {nits_con_valor.str.contains(r"\\.", na=False).sum():,}')
print(f'Con guiones: {nits_con_valor.str.contains("-", na=False).sum():,}')

# Distribucion por longitud
nits_solo_digitos = nits_con_valor.str.replace(r'[^0-9]', '', regex=True)
print(f'\nDistribucion por longitud (solo digitos):')
print(nits_solo_digitos.str.len().value_counts().sort_index())

ANALISIS DE FORMATO DEL NIT
Registros con NIT informado: 11,134
Solo digitos: 0
Con comas: 0
Con puntos: 0
Con guiones: 0

Distribucion por longitud (solo digitos):
NIT
0        5
7        3
8      332
9     3154
10    3823
11    3817
Name: count, dtype: int64


In [37]:
print('LIMPIEZA DEL CAMPO NIT')
print('=' * 60)

# Crear columna con solo digitos
df_camara['NIT_LIMPIO'] = df_camara['NIT'].astype(str).str.replace(r'[^0-9]', '', regex=True)

# Marcar NITs validos (excluir "No aplica", vacios y ceros)
df_camara['NIT_VALIDO'] = (
    ~df_camara['NIT'].isin(VALORES_NULOS) &
    (df_camara['NIT_LIMPIO'].str.len() > 0) &
    (df_camara['NIT_LIMPIO'] != '0')
)

print(f'NITs validos: {df_camara["NIT_VALIDO"].sum():,}')
print(f'NITs invalidos: {(~df_camara["NIT_VALIDO"]).sum():,}')

# Duplicados
nits_validos_df = df_camara[df_camara['NIT_VALIDO']]
duplicados = nits_validos_df['NIT_LIMPIO'].duplicated(keep=False).sum()
print(f'\nRegistros con NIT duplicado: {duplicados:,}')
print(f'NITs unicos validos: {nits_validos_df["NIT_LIMPIO"].nunique():,}')

print(f'\nMuestra - NIT original vs NIT limpio:')
display(df_camara[df_camara['NIT_VALIDO']][['NIT', 'NIT_LIMPIO', 'RAZON SOCIAL']].head(10))

LIMPIEZA DEL CAMPO NIT
NITs validos: 11,129
NITs invalidos: 9,151

Registros con NIT duplicado: 6
NITs unicos validos: 11,126

Muestra - NIT original vs NIT limpio:


,NIT,NIT_LIMPIO,RAZON SOCIAL
3,58181062,58181062,TOCORA QUINTERO ORLANDO
4,9020197540,9020197540,ASOCIACIÓN UNION CAMPESINA DEL TOLIMA
5,9020197952,9020197952,ASOCIACIÓN HISTORIAS QUE DEJAN HUELLA
6,9020191867,9020191867,"CORPORACIÓN AGROAMBIENTAL ECOSOSTENIBLE ""CORECA"""
7,9020186000,9020186000,AGREMIACION CAMPESINA RAICES DEL TOLIMA
8,9020185643,9020185643,ASOCIACION DE MOTOCARROS LA MIEL
9,9020188766,9020188766,FUNDACION HELICONIAS: FUERZA QUE FLORECE
10,9020188734,9020188734,FUNDACION PARA LA NOVIOLENCIA Y LA CULTURA DE PAZ JUNTOS
11,9020187632,9020187632,ASOCIACIÓN DE MUJERES Y JOVENES EMPODERADOS DEL CAMPO
12,9020177978,9020177978,ASOCIACIÓN DE MODAS MARY


---
## 4. Análisis de Variables Clave para el Proyecto

Analizamos las columnas más relevantes para la clasificación de empresas y el posterior análisis financiero: tipo de organización, tamaño empresarial, grupo NIIF, municipio y estado de la matrícula.

In [38]:
print('DISTRIBUCION POR TIPO DE ORGANIZACION')
print('=' * 60)
print(df_camara['ORGANIZACION'].value_counts())

print(f'\nDISTRIBUCION POR CATEGORIA')
print('=' * 60)
print(df_camara['CATEGORIA'].value_counts())

DISTRIBUCION POR TIPO DE ORGANIZACION
ORGANIZACION
ESTABLECIMIENTO DE COMERCIO           9001
PERSONA NATURAL                       7443
ESAL                                  1864
S.A.S                                 1599
SOCIEDAD LIMITADA                      139
ECONOMIA SOLIDARIA                     116
SOCIEDAD ANONIMA                        86
SOCIEDAD EN COMANDITA                   16
EXTRANJERAS                              4
UNIPERSONALES                            3
SOCIEDAD CIVIL                           2
SOCIEDAD EN COMANDITA POR ACCIONES       1
ASOCIATIVAS DE TRABAJO                   1
Name: count, dtype: int64

DISTRIBUCION POR CATEGORIA
CATEGORIA
PRINCIPAL    20147
AGENCIA        103
SUCURSAL        25
Name: count, dtype: int64


In [40]:
print('DISTRIBUCION POR GRUPO NIIF')
print('=' * 60)
print(df_camara['GRUPO NIIF'].value_counts())

DISTRIBUCION POR GRUPO NIIF
GRUPO NIIF
GRUPO III - MICROEMPRESAS                         10661
No aplica                                          9148
GRUPO II                                            437
GRUPO I - NIIF PLENAS                                14
RESOLUCION 414/2014                                  10
ENTIDADES PUBLICAS ART. 2 RES. 743 / 2013             3
DECRETO 2649/1993 - SUPERSALUD Y SUPERSUBSIDIO        2
Name: count, dtype: int64


In [41]:
print('ESTADO DE LA MATRICULA')
print('=' * 60)
print(df_camara['ESTADO DE LA MATRICULA'].value_counts())

print(f'\nMUNICIPIOS COMERCIALES (Top 20)')
print('=' * 60)
print(df_camara['MUNICIPIO COMERCIAL'].value_counts().head(20))

ESTADO DE LA MATRICULA
ESTADO DE LA MATRICULA
MA    18295
IA     1980
Name: count, dtype: int64

MUNICIPIOS COMERCIALES (Top 20)
MUNICIPIO COMERCIAL
73001 - IBAGUE    20275
Name: count, dtype: int64


---
## 5. Carga Completa de Datasets SIREM

Los datos financieros provienen del **Sistema de Información y Reporte Empresarial (SIREM)** de la Superintendencia de Sociedades. Son 4 archivos CSV que contienen estados financieros bajo normas NIIF de empresas colombianas.

Todos los datasets comparten la misma estructura de 11 columnas en **formato vertical** (una fila por cada par concepto-valor), donde cada fila representa un concepto contable específico con su valor numérico para una empresa y periodo determinado.

| Dataset | Tamaño | Contenido |
|---------|--------|-----------|
| Carátula | 2.07 GB | Metadatos de empresas y reportes |
| Estado de Situación Financiera | 4.10 GB | Balance General |
| Estado de Resultado Integral | 1.57 GB | Estado de Pérdidas y Ganancias |
| Estado de Flujo de Efectivo | 1.43 GB | Movimientos de efectivo |

In [42]:
ARCHIVOS_SIREM = {
    'caratula': 'Estados_Financieros_NIIF-_Carátula_20260204.csv',
    'situacion_financiera': 'Estados_Financieros_NIIF-_Estado_de_Situación_Financiera_20260203.csv',
    'resultado_integral': 'Estados_Financieros_NIIF-_Estado_de_Resultado_Integral_20260204.csv',
    'flujo_efectivo': 'Estados_Financieros_NIIF-_Estado_de_Flujo_Efectivo_20260204.csv'
}

print('ARCHIVOS SIREM')
print('=' * 60)
for nombre, archivo in ARCHIVOS_SIREM.items():
    ruta_archivo = BASE_PATH / archivo
    tamano_gb = ruta_archivo.stat().st_size / (1024**3)
    print(f'  {nombre}: {tamano_gb:.2f} GB')

ARCHIVOS SIREM
  caratula: 2.07 GB
  situacion_financiera: 4.10 GB
  resultado_integral: 1.57 GB
  flujo_efectivo: 1.43 GB


In [43]:
print('CARGA COMPLETA DE DATASETS SIREM')
print('=' * 60)

df_caratula = pd.read_csv(BASE_PATH / ARCHIVOS_SIREM['caratula'], encoding='utf-8', low_memory=False)
print(f'[OK] Caratula: {len(df_caratula):,} filas')

df_situacion = pd.read_csv(BASE_PATH / ARCHIVOS_SIREM['situacion_financiera'], encoding='utf-8', low_memory=False)
print(f'[OK] Situacion Financiera: {len(df_situacion):,} filas')

df_resultado = pd.read_csv(BASE_PATH / ARCHIVOS_SIREM['resultado_integral'], encoding='utf-8', low_memory=False)
print(f'[OK] Resultado Integral: {len(df_resultado):,} filas')

df_flujo = pd.read_csv(BASE_PATH / ARCHIVOS_SIREM['flujo_efectivo'], encoding='utf-8', low_memory=False)
print(f'[OK] Flujo de Efectivo: {len(df_flujo):,} filas')

datasets_sirem = {
    'Caratula': df_caratula,
    'Situacion Financiera': df_situacion,
    'Resultado Integral': df_resultado,
    'Flujo de Efectivo': df_flujo
}

total_filas = sum(len(ds) for ds in datasets_sirem.values())
print(f'\nTotal filas SIREM: {total_filas:,}')
print(f'Columnas: {list(df_situacion.columns)}')

CARGA COMPLETA DE DATASETS SIREM
[OK] Caratula: 8,685,453 filas
[OK] Situacion Financiera: 17,851,220 filas
[OK] Resultado Integral: 7,320,752 filas
[OK] Flujo de Efectivo: 5,769,293 filas

Total filas SIREM: 39,626,718
Columnas: ['CODIGO_INSTANCIA', 'NIT', 'NUMERO_RADICADO', 'ID_PUNTO_ENTRADA', 'PUNTO_ENTRADA', 'ID_TAXONOMIA', 'TAXONOMIA', 'FECHA_CORTE', 'CONCEPTO', 'PERIODO', 'VALOR']


---
## 6. Limpieza del NIT en Datasets SIREM

El campo NIT en los datasets SIREM utiliza **comas como separador de miles** (ej: `"830,010,665"`). Para poder realizar el cruce con la Cámara de Comercio, es necesario eliminar estos separadores y dejar el NIT como una cadena de solo dígitos.

In [44]:
print('FORMATO DEL NIT EN SIREM (ANTES DE LIMPIEZA)')
print('=' * 60)

for nombre, ds in datasets_sirem.items():
    nit_col = ds['NIT'].astype(str)
    con_comas = nit_col.str.contains(',', na=False).sum()
    solo_digitos = nit_col.str.match(r'^\d+$', na=False).sum()
    
    print(f'\n>>> {nombre}:')
    print(f'  Registros con comas: {con_comas:,} de {len(ds):,} ({con_comas/len(ds)*100:.0f}%)')
    print(f'  Solo digitos: {solo_digitos:,}')
    print(f'  Ejemplo: {nit_col.iloc[0]}')

FORMATO DEL NIT EN SIREM (ANTES DE LIMPIEZA)

>>> Caratula:
  Registros con comas: 8,685,453 de 8,685,453 (100%)
  Solo digitos: 0
  Ejemplo: 900,152,104

>>> Situacion Financiera:
  Registros con comas: 17,851,220 de 17,851,220 (100%)
  Solo digitos: 0
  Ejemplo: 830,010,665

>>> Resultado Integral:
  Registros con comas: 7,320,752 de 7,320,752 (100%)
  Solo digitos: 0
  Ejemplo: 860,353,641

>>> Flujo de Efectivo:
  Registros con comas: 5,769,293 de 5,769,293 (100%)
  Solo digitos: 0
  Ejemplo: 901,403,004


In [45]:
print('LIMPIEZA DE NIT EN DATASETS SIREM')
print('=' * 60)

for nombre, ds in datasets_sirem.items():
    ds['NIT_LIMPIO'] = ds['NIT'].astype(str).str.replace(r'[^0-9]', '', regex=True)
    
    longitudes = ds['NIT_LIMPIO'].str.len().value_counts().sort_index()
    nits_unicos = ds['NIT_LIMPIO'].nunique()
    
    print(f'\n>>> {nombre}:')
    print(f'  NITs unicos: {nits_unicos:,}')
    print(f'  Longitudes: {longitudes.to_dict()}')
    print(f'  Ejemplo: {ds["NIT"].iloc[0]}  ->  {ds["NIT_LIMPIO"].iloc[0]}')

LIMPIEZA DE NIT EN DATASETS SIREM

>>> Caratula:
  NITs unicos: 43,041
  Longitudes: {9: 8685453}
  Ejemplo: 900,152,104  ->  900152104

>>> Situacion Financiera:
  NITs unicos: 43,025
  Longitudes: {9: 17851220}
  Ejemplo: 830,010,665  ->  830010665

>>> Resultado Integral:
  NITs unicos: 43,037
  Longitudes: {9: 7320752}
  Ejemplo: 860,353,641  ->  860353641

>>> Flujo de Efectivo:
  NITs unicos: 42,895
  Longitudes: {9: 5769293}
  Ejemplo: 901,403,004  ->  901403004


---
## 7. Primer Intento de Cruce (JOIN) por NIT

Intentamos el cruce directo entre los NITs limpios de ambas fuentes para verificar cuántas empresas de Ibagué tienen datos financieros en SIREM.

In [46]:
print('INTENTO DE MATCH DIRECTO')
print('=' * 60)

nits_camara = set(df_camara[df_camara['NIT_VALIDO']]['NIT_LIMPIO'].unique())

print(f'\nCamara de Comercio - NITs unicos validos: {len(nits_camara):,}')
print(f'Muestra Camara: {list(nits_camara)[:5]}')

nits_sirem = set(df_situacion['NIT_LIMPIO'].unique())
print(f'\nSIREM (Sit. Financiera) - NITs unicos: {len(nits_sirem):,}')
print(f'Muestra SIREM: {list(nits_sirem)[:5]}')

match_directo = nits_camara & nits_sirem
print(f'\nCoincidencias directas: {len(match_directo)}')
print(f'\n>>> El match directo falla. Se requiere investigar la causa.')

INTENTO DE MATCH DIRECTO

Camara de Comercio - NITs unicos validos: 11,126
Muestra Camara: ['11045447928', '286888713', '934107796', '11105808859', '9017939907']

SIREM (Sit. Financiera) - NITs unicos: 43,025
Muestra SIREM: ['900138972', '900216619', '830016966', '800163101', '900418252']

Coincidencias directas: 0

>>> El match directo falla. Se requiere investigar la causa.


---
## 8. Diagnóstico: Incompatibilidad en el Formato del NIT

El cruce directo produce **0 coincidencias**, lo cual indica una incompatibilidad en el formato del NIT entre ambas fuentes. Al analizar las longitudes, se observa que:

- **SIREM**: todos los NITs tienen exactamente **9 dígitos** (NIT base, sin dígito de verificación)
- **Cámara de Comercio**: los NITs tienen longitudes variables (7 a 11 dígitos)

Para entender esta discrepancia, analizamos qué tipo de organización corresponde a cada longitud de NIT en la Cámara de Comercio.

In [47]:
print('TIPO DE ORGANIZACION POR LONGITUD DE NIT')
print('=' * 60)

camara_validos = df_camara[df_camara['NIT_VALIDO']].copy()
camara_validos['NIT_LEN'] = camara_validos['NIT_LIMPIO'].str.len()

for longitud in sorted(camara_validos['NIT_LEN'].unique()):
    subset = camara_validos[camara_validos['NIT_LEN'] == longitud]
    print(f'\n>>> {longitud} digitos ({len(subset):,} registros):')
    print(subset['ORGANIZACION'].value_counts().to_string())

TIPO DE ORGANIZACION POR LONGITUD DE NIT

>>> 7 digitos (3 registros):
ORGANIZACION
PERSONA NATURAL    3

>>> 8 digitos (332 registros):
ORGANIZACION
PERSONA NATURAL    332

>>> 9 digitos (3,154 registros):
ORGANIZACION
PERSONA NATURAL          3152
ESAL                        1
SOCIEDAD EN COMANDITA       1

>>> 10 digitos (3,823 registros):
ORGANIZACION
ESAL                           1860
S.A.S                          1569
ECONOMIA SOLIDARIA              116
PERSONA NATURAL                 114
SOCIEDAD LIMITADA               101
SOCIEDAD ANONIMA                 37
SOCIEDAD EN COMANDITA            15
EXTRANJERAS                       4
UNIPERSONALES                     3
SOCIEDAD CIVIL                    2
ESTABLECIMIENTO DE COMERCIO       1
ASOCIATIVAS DE TRABAJO            1

>>> 11 digitos (3,817 registros):
ORGANIZACION
PERSONA NATURAL    3817


### Hallazgo Clave: Dígito de Verificación (DV)

El análisis revela un patrón claro:

| Longitud | Tipo de Registro | Identificador | Cantidad |
|----------|-----------------|---------------|----------|
| 7-8 dígitos | Persona Natural | Cédula de ciudadanía | 335 |
| 9 dígitos | Persona Natural (99.9%) | Cédula de ciudadanía | 3,154 |
| **10 dígitos** | **Empresas (SAS, Ltda, SA, ESAL)** | **NIT + Dígito de Verificación** | **3,823** |
| 11 dígitos | Persona Natural | Cédula de ciudadanía | 3,817 |

**Causa raíz del mismatch**: En Colombia, el NIT empresarial tiene 9 dígitos base más un dígito de verificación (DV). La Cámara de Comercio almacena el NIT completo (9 + 1 = **10 dígitos**), mientras que SIREM almacena solo el NIT base (**9 dígitos**, sin DV).

**Ejemplo**:
- Cámara de Comercio: `8907022981` (10 dígitos, con DV = `1`)
- SIREM: `890702298` (9 dígitos, sin DV)

**Solución**: Para los NITs de 10 dígitos, eliminar el último dígito (DV) para obtener el NIT base de 9 dígitos compatible con SIREM.

In [48]:
print('NORMALIZACION DEL NIT PARA JOIN')
print('=' * 60)

# Para NITs de 10 digitos: quitar el ultimo digito (digito de verificacion)
# Para el resto: mantener tal cual (cedulas de personas naturales)
camara_validos['NIT_PARA_JOIN'] = camara_validos['NIT_LIMPIO'].apply(
    lambda x: x[:-1] if len(x) == 10 else x
)

print('Distribucion de longitud del NIT normalizado (NIT_PARA_JOIN):')
print(camara_validos['NIT_PARA_JOIN'].str.len().value_counts().sort_index())

print(f'\nEjemplos de la transformacion (NITs de 10 digitos):')
muestra = camara_validos[camara_validos['NIT_LEN'] == 10][['NIT', 'NIT_LIMPIO', 'NIT_PARA_JOIN', 'RAZON SOCIAL']].head(10)
display(muestra)

NORMALIZACION DEL NIT PARA JOIN
Distribucion de longitud del NIT normalizado (NIT_PARA_JOIN):
NIT_PARA_JOIN
7        3
8      332
9     6977
11    3817
Name: count, dtype: int64

Ejemplos de la transformacion (NITs de 10 digitos):


,NIT,NIT_LIMPIO,NIT_PARA_JOIN,RAZON SOCIAL
4,9020197540,9020197540,902019754,ASOCIACIÓN UNION CAMPESINA DEL TOLIMA
5,9020197952,9020197952,902019795,ASOCIACIÓN HISTORIAS QUE DEJAN HUELLA
6,9020191867,9020191867,902019186,"CORPORACIÓN AGROAMBIENTAL ECOSOSTENIBLE ""CORECA"""
7,9020186000,9020186000,902018600,AGREMIACION CAMPESINA RAICES DEL TOLIMA
8,9020185643,9020185643,902018564,ASOCIACION DE MOTOCARROS LA MIEL
9,9020188766,9020188766,902018876,FUNDACION HELICONIAS: FUERZA QUE FLORECE
10,9020188734,9020188734,902018873,FUNDACION PARA LA NOVIOLENCIA Y LA CULTURA DE PAZ JUNTOS
11,9020187632,9020187632,902018763,ASOCIACIÓN DE MUJERES Y JOVENES EMPODERADOS DEL CAMPO
12,9020177978,9020177978,902017797,ASOCIACIÓN DE MODAS MARY
13,9020184684,9020184684,902018468,FUNDACIÓN URBANA DE TRANSFORMACION UNIVERSAL RENOVACIÓN...


---
## 9. Validación del Cruce con NIT Normalizado

Con el NIT normalizado (sin dígito de verificación para empresas de 10 dígitos), realizamos nuevamente el cruce contra los 4 datasets SIREM completos.

In [49]:
print('RESULTADO DEL JOIN CON NIT NORMALIZADO')
print('=' * 60)

nits_camara_join = set(camara_validos['NIT_PARA_JOIN'].unique())

for nombre, ds in datasets_sirem.items():
    nits_ds = set(ds['NIT_LIMPIO'].unique())
    match = nits_camara_join & nits_ds
    print(f'  {nombre}: {len(match):,} empresas de Ibague encontradas')

# Match global
nits_todos_sirem = set()
for ds in datasets_sirem.values():
    nits_todos_sirem.update(ds['NIT_LIMPIO'].unique())

match_total = nits_camara_join & nits_todos_sirem
print(f'\n  TOTAL empresas de Ibague con datos en SIREM: {len(match_total):,}')

RESULTADO DEL JOIN CON NIT NORMALIZADO
  Caratula: 66 empresas de Ibague encontradas
  Situacion Financiera: 66 empresas de Ibague encontradas
  Resultado Integral: 66 empresas de Ibague encontradas
  Flujo de Efectivo: 65 empresas de Ibague encontradas

  TOTAL empresas de Ibague con datos en SIREM: 66


In [50]:
print('EMPRESAS DE IBAGUE CON DATOS FINANCIEROS EN SIREM')
print('=' * 60)

nits_match = nits_camara_join & set(df_situacion['NIT_LIMPIO'].unique())

df_empresas_match = camara_validos[camara_validos['NIT_PARA_JOIN'].isin(nits_match)]

col_tamano = [c for c in df_camara.columns if 'TAMA' in c and 'EMPRESA' in c]
col_t = col_tamano[0] if col_tamano else None

columnas_display = ['NIT_PARA_JOIN', 'RAZON SOCIAL', 'ORGANIZACION', 'MUNICIPIO COMERCIAL', 'GRUPO NIIF']
if col_t:
    columnas_display.append(col_t)

print(f'\nTotal empresas con datos financieros: {len(df_empresas_match["NIT_PARA_JOIN"].unique()):,}')

print(f'\nDistribucion por tipo de organizacion:')
print(df_empresas_match['ORGANIZACION'].value_counts())

print(f'\nListado completo:')
display(df_empresas_match[columnas_display].drop_duplicates('NIT_PARA_JOIN').reset_index(drop=True))

EMPRESAS DE IBAGUE CON DATOS FINANCIEROS EN SIREM

Total empresas con datos financieros: 66

Distribucion por tipo de organizacion:
ORGANIZACION
S.A.S                          38
SOCIEDAD LIMITADA              12
SOCIEDAD ANONIMA               12
SOCIEDAD EN COMANDITA           2
ESTABLECIMIENTO DE COMERCIO     1
EXTRANJERAS                     1
Name: count, dtype: int64

Listado completo:


,NIT_PARA_JOIN,RAZON SOCIAL,ORGANIZACION,MUNICIPIO COMERCIAL,GRUPO NIIF,CIIU TAMAÑO EMPRESARIAL
0,890702298,AGROPECUARIA CALICANTO S.A.S.,S.A.S,73001 - IBAGUE,GRUPO II,L6810
1,809002625,INTERNACIONAL DE ELECTRICOS S.A.S.,S.A.S,73001 - IBAGUE,GRUPO I - NIIF PLENAS,G4663
2,890700062,LAS PIZARRAS SAS,S.A.S,73001 - IBAGUE,GRUPO II,K6613
3,809004254,GRUPO INVERSIONES MAR S.A.S,S.A.S,73001 - IBAGUE,GRUPO III - MICROEMPRESAS,L6810
4,809002262,CLORQUIMICOS LTDA,SOCIEDAD LIMITADA,73001 - IBAGUE,GRUPO II,G4664
5,809001726,FUNERALES LA VERDE ESPERANZA S.A.S.,S.A.S,73001 - IBAGUE,GRUPO II,S9603
6,809001092,IMS S.A.S.,S.A.S,73001 - IBAGUE,GRUPO II,N8121
7,809000876,TEMPORALES UNOA S.A.S,S.A.S,73001 - IBAGUE,GRUPO II,N7820
8,860354473,PROMOCIONES Y COBRANZAS BETA S.A.,SOCIEDAD ANONIMA,73001 - IBAGUE,No aplica,No reporta
9,809000514,CONSTRUVIAL S.A.S,S.A.S,73001 - IBAGUE,GRUPO II,F4210


In [51]:
print('VERIFICACION DE LOS MATCHES')
print('=' * 60)

# Todas son de Ibague?
print(f'\nMunicipio de las empresas encontradas:')
print(df_empresas_match['MUNICIPIO COMERCIAL'].value_counts())

# Cuantas son NIIF Pymes en SIREM?
sirem_pymes = df_situacion[df_situacion['PUNTO_ENTRADA'].str.contains('Pymes', case=False, na=False)]
nits_sirem_pymes = set(sirem_pymes['NIT_LIMPIO'].unique())
match_pymes = nits_camara_join & nits_sirem_pymes
print(f'\nEmpresas NIIF Pymes (relevantes para la tesis): {len(match_pymes):,} de {len(nits_match):,}')

# Verificar un match concreto
ejemplo_nit = list(nits_match)[:3]
print(f'\nVerificacion cruzada de 3 empresas:')
for nit in ejemplo_nit:
    camara_row = df_empresas_match[df_empresas_match['NIT_PARA_JOIN'] == nit].iloc[0]
    sirem_filas = df_situacion[df_situacion['NIT_LIMPIO'] == nit]
    print(f'\n  NIT: {nit}')
    print(f'  Camara: {camara_row["RAZON SOCIAL"]} (NIT original: {camara_row["NIT"]})')
    print(f'  SIREM: {len(sirem_filas):,} registros financieros')
    print(f'  Punto entrada: {sirem_filas["PUNTO_ENTRADA"].unique()}')

VERIFICACION DE LOS MATCHES

Municipio de las empresas encontradas:
MUNICIPIO COMERCIAL
73001 - IBAGUE    66
Name: count, dtype: int64

Empresas NIIF Pymes (relevantes para la tesis): 61 de 66

Verificacion cruzada de 3 empresas:

  NIT: 890701760
  Camara: JOSE I. DIAZ M. Y CIA. DIAGROTOL S. EN C. (NIT original: 8907017607)
  SIREM: 643 registros financieros
  Punto entrada: ['40 NIIF Pymes - Individuales' '40 NIIF Pymes - Individual Grupo 2']

  NIT: 800043005
  Camara: PROCESADORA DE CEREALES DE COLOMBIA S.A.S (NIT original: 8000430053)
  SIREM: 394 registros financieros
  Punto entrada: ['40 NIIF Pymes - Individuales' '40 NIIF Pymes - Individual Grupo 2']

  NIT: 800244693
  Camara: SERVICIOS TECNICOS Y PROFESIONALES DE COLOMBIA LTDA EN LIQUIDACION (NIT original: 8002446933)
  SIREM: 36 registros financieros
  Punto entrada: ['10 NIIF Plenas - Individuales']


---
## 10. Conclusiones del ETL

### Hallazgos principales

1. **Cámara de Comercio de Ibagué**: 20,280 registros totales, de los cuales 11,129 tienen NIT válido. Los 9,146 registros sin NIT corresponden principalmente a **establecimientos de comercio** (sucursales) que operan bajo el NIT de su empresa propietaria.

2. **Incompatibilidad de NIT resuelta**: Se identificó que la Cámara de Comercio almacena el NIT con **dígito de verificación** (10 dígitos), mientras que SIREM usa el NIT base (9 dígitos). La solución fue eliminar el último dígito de los NITs de 10 dígitos.

3. **Resultado del cruce**: Se identificaron **66 empresas de Ibagué** con datos financieros completos en SIREM. De estas, **61 son NIIF Pymes**, directamente relevantes para la tesis.

4. **Composición de las 66 empresas**: SAS (38), Sociedades Limitadas (12), Sociedades Anónimas (12), Sociedades en Comandita (2), otras (2). Todas ubicadas en el municipio 73001 - Ibagué.

5. **Volumen de datos por empresa**: Cada empresa tiene entre 200 y 400+ registros financieros en el Balance General, cubriendo múltiples años y conceptos contables, lo cual es suficiente para el cálculo de indicadores y el entrenamiento del modelo de ML.

### Limitación identificada

Solo 66 de las 1,733 sociedades comerciales registradas en Ibagué aparecen en SIREM. Esto se debe a que **SIREM solo contiene empresas vigiladas por la Superintendencia de Sociedades**, y la mayoría de PYMES no están obligadas a reportar. Esta limitación está documentada en la propuesta de trabajo de grado.

### Próximos pasos

1. Ejecutar el JOIN completo para extraer los datos financieros de las 66 empresas
2. Pivotar los datos de formato vertical a horizontal (una columna por concepto contable)
3. Calcular los indicadores financieros (liquidez, rentabilidad, endeudamiento, eficiencia)
4. Crear las etiquetas de riesgo para el modelo de Machine Learning

In [52]:
print('=' * 70)
print('RESUMEN FINAL DEL ETL')
print('=' * 70)

print(f'\nFUENTES DE DATOS:')
print(f'  Camara de Comercio: {len(df_camara):,} registros, {len(df_camara.columns)} columnas')
for nombre, ds in datasets_sirem.items():
    print(f'  SIREM - {nombre}: {len(ds):,} filas')

print(f'\nCALIDAD DE DATOS - CAMARA DE COMERCIO:')
print(f'  NITs validos: {df_camara["NIT_VALIDO"].sum():,}')
print(f'  NITs invalidos (No aplica / vacios): {(~df_camara["NIT_VALIDO"]).sum():,}')
print(f'  NITs unicos validos: {nits_validos_df["NIT_LIMPIO"].nunique():,}')

print(f'\nRESULTADO DEL CRUCE:')
print(f'  Empresas de Ibague con datos financieros en SIREM: {len(nits_match):,}')
print(f'  De las cuales son NIIF Pymes: {len(match_pymes):,}')
print(f'  Campo de cruce: NIT_PARA_JOIN (9 digitos, sin DV) = NIT_LIMPIO SIREM (9 digitos)')

RESUMEN FINAL DEL ETL

FUENTES DE DATOS:
  Camara de Comercio: 20,280 registros, 87 columnas
  SIREM - Caratula: 8,685,453 filas
  SIREM - Situacion Financiera: 17,851,220 filas
  SIREM - Resultado Integral: 7,320,752 filas
  SIREM - Flujo de Efectivo: 5,769,293 filas

CALIDAD DE DATOS - CAMARA DE COMERCIO:
  NITs validos: 11,129
  NITs invalidos (No aplica / vacios): 9,151
  NITs unicos validos: 11,126

RESULTADO DEL CRUCE:
  Empresas de Ibague con datos financieros en SIREM: 66
  De las cuales son NIIF Pymes: 61
  Campo de cruce: NIT_PARA_JOIN (9 digitos, sin DV) = NIT_LIMPIO SIREM (9 digitos)
